In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import trange
from konlpy.tag import Twitter

In [19]:
def Korean_tagger(Text):
    twitter = Twitter()
    try:
        malist = twitter.pos(Text, norm=True, stem=True)
        r = []
        remove_tag =['Eomi','PreEomi','Josa','Determiner','Foreign','Alpha','Number','Punctuation','Suffix','Unknown','Hashtag','KoreanParticle','ScreenName']

        stopwords = sum_stopwords


        for word in malist:
            # 어미/조사/구두점/ㅋㅋ^^ㅎㅎ/음표살림/Alphabet/부사는 대상에서 제외
            if not word[1] in remove_tag:
                if not word[0] in r:
                    if not word[0] in stopwords:
                       # 숫자, 특수문자 제거.
                        r.append(word[0])
        return ' '.join(r)
    except Exception as e:
        return ''

In [20]:
naver_user = pd.read_csv('[Final]naver_user_table.csv',sep='\t',encoding='utf-8')
dbdbdeep_user = pd.read_csv('[Final]dbdbdeep_user_table.csv',sep='\t',encoding='utf-8')

In [21]:
Total = pd.concat([naver_user,dbdbdeep_user])

In [22]:
from sklearn.utils import shuffle
Total = shuffle(Total)

In [23]:
Blog_infro_text = Total['Blog_info_text'].tolist()

In [24]:
Blog_infro_text = list(map(lambda x : Korean_tagger(x),Blog_infro_text))

/Users/yonggeol/anaconda3/envs/py/lib/python3.5/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


## TFIDF model 저장 & load

## model구축

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
v_title = TfidfVectorizer()
x = v_title.fit_transform(Title)

## model저장

In [ ]:
import pickle
#Save vectorizer.vocabulary_
pickle.dump(v_title.vocabulary_,open("Title_feature.pkl","wb"))

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()

In [ ]:
loaded_vec = TfidfVectorizer(decode_error="replace",vocabulary=pickle.load(open("Title_feature.pkl", "rb")))
tfidf = transformer.fit_transform(loaded_vec.fit_transform(np.array(["안녕하세요"])))

In [ ]:
x = Total['Embedding_text'].apply(lambda x : transformer.fit_transform(loaded_vec.fit_transform(np.array([x]))))

In [ ]:
Array = list(map(lambda x : x.toarray(),x))

In [ ]:
Label = Total['label']

In [ ]:
test = pd.DataFrame({'Array':Array,'label':Label})

In [ ]:
x = test['Array'].apply(lambda x: x.flatten())
x = x.tolist()

In [ ]:
y = test['label']

# Doc2vec model 저장 & load

## model 구축

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
sentences = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(text)]

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [ ]:
doc_vectorizer = Doc2Vec(
    dm=0,            # PV-DBOW / default 1
    dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=8,        # distance between the predicted word and context words
    size=92,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    min_count=20,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    workers=cores,   # multi cpu
    hs = 1,          # hierarchical softmax / default 0
    negative = 10,   # negative sampling / default 5
)

In [ ]:
doc_vectorizer.build_vocab(sentences)

In [ ]:
print(str(doc_vectorizer))

In [ ]:
doc_vectorizer.corpus_count*0.03

In [ ]:
doc_vectorizer.iter

# model 학습

In [ ]:
import time
start = time.time()
for epoch in range(10):
    doc_vectorizer.train(sentences, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay
end = time.time()
print("During Time: {}".format(end-start))

In [ ]:
model_name = 'Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model'
doc_vectorizer.save(model_name)

In [ ]:
doc_vectorizer = Doc2Vec.load(model_name)